# Fusion des fichiers logs et logs error

### L'objectif est d'observer le nombre d'opérations réussies et d'erreur par heure, sur la période de temps disponible (aout-novembre 24)

#### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Import et préparation du fichier log error

In [ ]:
# Ouverture du fichier
df_error = pd.read_csv('../Data/logEtl/241016_LogETLError.csv', sep=';', dtype={'Program_Id': str, 'Schedules_Id': str, 'Schedules_Name': str})

In [ ]:
df_error.head()

In [ ]:
# Conversion de la colonne ETL_StartDateTime en Datetime pandas
df_error['ETL_StartDateTime']=pd.to_datetime(df_error['ETL_StartDateTime'], format="%Y-%m-%d %H:%M:%S")

In [ ]:
# Récupération de la date de début et de fin du fichier

start_date = df_error['ETL_StartDateTime'].min()
end_date = df_error['ETL_StartDateTime'].max()

print("Date la plus ancienne :", start_date)
print("Date la plus récente :", end_date)

In [ ]:
# Ajout d'une colonne Date et heure ne tenant pas compte des minutes
df_error["Date et heure"] = df_error["ETL_StartDateTime"].dt.floor("h")

In [ ]:
df_error.head()

## Catégorisation des messages d'erreur

In [ ]:
# Lister toutes les valeurs distinctes dans la colonne 'Message'
distinct_types = df_error['Message'].unique()
print(distinct_types)
print(len(distinct_types))

In [ ]:
import pandas as pd

def categorize_message(message):
    message = message.lower()
    if "Guidez Atelier".lower() in message or "Guidez Social".lower()  in message:
        return "Guidez Atelier Error"
    elif "La taille du champ".lower()  in message:
        return "Field Size Error" 
    elif "Un incident est survenu Object reference not set to an instance of an object.".lower()  in message:
        return "Object Reference Error"
    elif "La valeur #REF!".lower()  in message:
        return "Value #REF! Error"
    elif "La valeur #VALUE!".lower()  in message:
        return "Value #VALUE! Error"
    elif "La valeur #N/A!".lower()  in message:
        return "Value #N/A! Error"
    elif "La valeur #DIV/0!".lower()  in message:
        return "Value #DIV/0! Error"
    elif "La valeur #NAME?".lower()  in message:
        return "Value #NAME? Error"
    elif "Connection Timeout Expired.  The timeout period elapsed during the post-login phase.".lower()  in message:
        return "Connection Timeout Error"
    elif "an error occurred during the pre-login handshake.".lower()  in message:
        return "Pre-Login Error"
    elif "Un incident est survenu L'instruction CREATE UNIQUE INDEX a été interrompue, car une clé dupliquée a été trouvée pour l'objet".lower()  in message:
        return "Unique Index Error"
    elif "Impossible de créer la table".lower()  in message:
        return "Create TABLE Error"
    elif "Un incident est survenu Échec de l'opération car un index ou des statistiques portant le nom".lower()  in message:
        return "Index Error (similar to unique index error)"
    elif "utilisé dans la clé primaire, sa valeur ne doit pas être vide".lower()  in message:
        return "Primary Key Error"
    elif "pas convertible en Heure".lower()  in message:
        return "Time Format Error"
    elif "Not a legal OleAut date.".lower()  in message or ("La date".lower()  in message and "est pas valide".lower()  in message):
        return "Date Format Error"
    elif "connexion au web service impossible".lower()  in message:
        return "Web Service Error"
    elif "impossible d'ouvrir la requête".lower()  in message and "Un incident est survenu".lower()  in message and "non valide".lower()  in message:
        return "SQL Invalid Query Error"
    elif "Impossible d\'ouvrir la requête SQL sur la connexion".lower()  in message and "Le délai d\'attente a été dépassé".lower()  in message:
        return "SQL Query Timeout Error"
    #elif "Le filtre de suppression n'est pas compatible SQL".lower()  in message:
    #    return "SQL Delete Filter Error"
    #elif "impossible d'ouvrir la requête sql".lower()  in message:
     #   return "SQL Other Error"
    elif "Le nom de colonne".lower()  in message and "n'existe pas dans la table ou la vue cible".lower()  in message:
        return "Column Name Error"
    else:
        return "Other Error"

df_error = df_error.assign(
    Message_Category=df_error["Message"].apply(categorize_message)
)

In [ ]:
df_error.head()

In [ ]:
message_count = df_error['Message_Category'].value_counts()
print(message_count)
print(len(message_count))

In [ ]:
import matplotlib.pyplot as plt

message_count = df_error['Message_Category'].value_counts()
plt.figure(figsize=(12, 8))
message_count.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title("Message Category Counts", fontsize=16)
plt.xlabel("Message Category", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Détail sur les messages catégorisés dans "Other Error"

In [ ]:

unique_other_errors = df_error[df_error['Message_Category'] == "Other Error"]['Message'].unique()
print(unique_other_errors)
print(len(unique_other_errors))

A retenir : dans "other error" il y a 4 messages d'erreurs différents.On les néglige pour l'instant

## Opération de Groupby 

In [ ]:
df_error.head()

In [ ]:
# choix des colonnes en vérifiant qu'il n'y ait pas de valeurs NaN (trop de problème dans la jointure après)
nan_count = df_error.isna().sum()
print(nan_count)

Je voulais utiliser Schedules_name, mais trop de valeurs NaN, donc je me rabats sur Program_Name. On va aussi aggréger sur Message_category évidement

In [ ]:
# liste des Message_Category
print(df_error["Message_Category"].unique())
print(df_error["Message_Category"].unique().shape)

In [ ]:
# liste des Program_Name
print(df_error["Program_Name"].unique())
print(df_error["Program_Name"].unique().shape)

In [ ]:
# Utiliser des crosstab (ou pivot) et joindre les DataFrames

# Comptage des catégories par Date et heure
df_cat = pd.crosstab(df_error["Date et heure"], df_error["Message_Category"])

# Comptage des programmes par Date et heure
df_prog = pd.crosstab(df_error["Date et heure"], df_error["Program_Name"])

# Joindre les deux sur l'index (qui est "Date et heure" dans les 2 crosstabs)
df_error_grouped2 = df_cat.join(df_prog, how="outer")

# Remettre "Date et heure" en colonne si besoin
df_error_grouped2 = df_error_grouped2.reset_index()

df_error_grouped2.head()

In [ ]:
# Vérifier les colonnes Error
error_columns = [col for col in df_error_grouped2.columns if "Error" in col]
print(error_columns)

In [ ]:
# sommer le nb d'erreurs dans les colonnes erreur pour la vérification
df_error_grouped2["Total_Errors"] = df_error_grouped2[error_columns].sum(axis=1)


In [ ]:
print(df_error_grouped2.shape)
df_error_grouped2.head()

## Ancienne opération Groupby de Guillaume

In [ ]:
# Group by la colonne Date et heure en faisant le nombre d'opérations
df_error_grouped = df_error.groupby("Date et heure").agg(
    nb_operations_error=("Message_Category", "count")
).reset_index()

In [ ]:
# Pourquoi????? 
#df_error_grouped = df_error_grouped.iloc[1:] 

In [ ]:
print(df_error_grouped.shape)
df_error_grouped.head()

## vérification

In [ ]:
result = df_error_grouped2.merge(df_error_grouped[["Date et heure", "nb_operations_error"]], on="Date et heure", how="inner")
result["Check"] = result["Total_Errors"] == result["nb_operations_error"]

In [ ]:
print(result.shape)
result.head()

In [ ]:
result_false_check = result[result["Check"] == False]
print(result_false_check["Date et heure"].unique())

Taille de resulat=false est de 0, c'est good

## Import et préparation du fichier log

In [ ]:
# Ouverture du fichier
#df = pd.read_parquet("/Users/guillaumeramirez/OneDrive - CentraleSupelec/Smart ETL - DOR/data/parquet/LogETL_20250130.csv_sub_2501301744.parquet")
#df = pd.read_parquet('/Users/Antoine/Library/CloudStorage/OneDrive-CentraleSupelec/Smart ETL - DOR/data/parquet/LogETL_20250130.csv_sub_2501301744.parquet')
df = pd.read_parquet("../../data/logParquet/LogETL_20250130.csv_sub_2501301744.parquet")

In [ ]:
df.head()

In [ ]:
# Conversion de la colonne ETL_StartDateTime en Datetime pandas
df['ETL_StartDateTime']=pd.to_datetime(df['ETL_StartDateTime'], format="%d/%m/%Y %H:%M")

In [ ]:
df.head()

In [ ]:
# Réduction du df pour qu'il soit dans la bonne période
df_reduced = df[(df['ETL_StartDateTime'] >= start_date) & (df['ETL_StartDateTime'] <= end_date)]

In [ ]:
df_reduced.head()

In [ ]:
# Ajout d'une colonne Date et heure ne tenant pas compte des minutes
#df_reduced["Date et heure"] = df_reduced["ETL_StartDateTime"].dt.floor("h")
df_reduced.loc[:, "Date et heure"] = df_reduced["ETL_StartDateTime"].dt.floor("h")

In [ ]:
df_reduced.head()

In [ ]:
df_reduced.shape

In [ ]:
df_grouped = df_reduced.groupby("Date et heure").agg(
    nb_operations=("Insert mode", "count"),  # Nombre total de lignes dans l'heure
    rows_added=("Rows added", "sum"),      # Somme des lignes ajoutées
    rows_updated=("Rows updated", "sum"),  # Somme des mises à jour
    rows_deleted=("Rows deleted", "sum")   # Somme des suppressions
).reset_index()

In [ ]:
df_grouped.head()

In [ ]:
df_grouped.shape

## Fusion des deux fichiers

In [ ]:
# Fusion des deux fichiers
df_final = df_error_grouped2.merge(
    df_grouped,
    on="Date et heure",
    how="outer",
)

# Changement des valeurs NaN en 0
df_final = df_final.fillna(0)

In [ ]:
df_final.head()

In [ ]:
df_final.shape

# Optionel pour visualisation

In [ ]:
df_filtered = df_final[df_final["Date et heure"].dt.date == pd.to_datetime("2024-10-03").date()]

df_filtered.head()

In [ ]:
# pour améliorer la visualisation, on va filtrer et retirer deux points aberrants

df_max_errors = df_final[df_final["Total_Errors"] == df_final["Total_Errors"].max()]
#df_max_errors.head()

df_filtered = df_final[~(df_final['Date et heure'] == '2024-08-29 11:00:00')]

df_max_errors = df_filtered[df_filtered["Total_Errors"] == df_filtered["Total_Errors"].max()]
df_max_errors.head()

df_filtered = df_filtered[~(df_filtered['Date et heure'] == '2024-10-03 17:00:00')]

In [ ]:
# Visualisation

# Création de la figure et des axes
fig, ax1 = plt.subplots(figsize=(12, 6))

# Axe principal (opérations normales)
ax1.plot(df_filtered["Date et heure"], df_filtered["nb_operations"], marker="o", linestyle="-", label="Nb opérations normales", color="blue")
ax1.set_xlabel("Heure")
ax1.set_ylabel("Nb opérations normales", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")

# Création d'un second axe Y pour les erreurs
ax2 = ax1.twinx()
ax2.plot(df_filtered["Date et heure"], df_filtered["Total_Errors"], marker="s", linestyle="--", label="Nb opérations erreurs", color="red")
ax2.set_ylabel("Nb opérations erreurs", color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Titre et légende
plt.title("Comparaison des opérations normales et erreurs par heure")
fig.tight_layout()

# Rotation des dates pour meilleure lisibilité
plt.xticks(rotation=45)

# Affichage du graphique
plt.show()


## Analyse des ETL les plus présents dans les 2 logs

In [ ]:
top_program = df_error['Program_Name'].value_counts()
top_program = top_program.head()
top_program.head()

In [ ]:
top_program = df['Program_Name'].value_counts()
top_program = top_program.head()
top_program.head()

# Fusion des DF de stat server

In [ ]:
df_reseau = pd.read_csv('../data/logServer/filtered/myreport_reseau_filtered.csv')
df_sql_statistic = pd.read_csv('../data/logServer/filtered/myreport_sql_statistic_filtered.csv')
df_sql_lock = pd.read_csv('../data/logServer/filtered/myreport_sql_lock_filtered.csv')
df_sql_general = pd.read_csv('../data/logServer/filtered/myreport_sql_general_filtered.csv')
df_ping = pd.read_csv('../data/logServer/filtered/myreport_ping_filtered.csv')
df_storage = pd.read_csv('../data/logServer/filtered/myreport_espace_disque_filtered.csv')
df_swap = pd.read_csv('../data/logServer/filtered/myreport_swap_filtered.csv')
df_sql_management_storage = pd.read_csv('../data/logServer/filtered/myreport_sql_gestionairedememoire_filtered.csv')
df_ram = pd.read_csv('../data/logServer/filtered/myreport_ram_filtered.csv')
df_cpu = pd.read_csv('../data/logServer/filtered/myreport_cpu_filtered.csv')

In [ ]:
df_cpu.head()

In [ ]:
dfs = [df_reseau, df_sql_statistic, df_sql_lock, df_sql_general, df_ping, df_storage, df_swap, df_sql_management_storage, df_ram, df_cpu]

In [ ]:
df_sql_general.head()

In [ ]:
from functools import reduce

df_server_stats = reduce(lambda left, right: pd.merge(left, right, on="Date et heure", how="outer"), dfs)

In [ ]:
df_server_stats.head()

In [ ]:
df_server_stats.columns

In [ ]:
df_server_stats_sansna = df_server_stats.dropna()

print(df_server_stats.shape)
print(df_server_stats_sansna.shape)

In [ ]:
df_server_stats.isna().sum().sum()

## Fusion du df conso logs + df conso stat server

In [ ]:
print(df_final["Date et heure"].dtype)
print(df_server_stats["Date et heure"].dtype)


In [ ]:
df_server_stats['Date et heure'] = pd.to_datetime(df_server_stats['Date et heure'])

In [ ]:
df_server_stats.head()

In [ ]:
print(df_final["Date et heure"].dtype)
print(df_server_stats["Date et heure"].dtype)
df_server_stats.shape

In [ ]:
# Fusion des deux df conso
df_global = df_final.merge(
    df_server_stats,
    on="Date et heure",
    how="outer",
)

In [ ]:
df_global.head()

In [ ]:
df_global.shape

In [ ]:
df_global.head()

In [ ]:
df_global = df_global.fillna(0)

In [ ]:
df_global.to_csv("../data/dataset_LogETL_LogServer.csv", index=False, encoding='utf-8')